# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re
import sys
import json

import nbformat

In [3]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import mbrain as mb

# Development

In [4]:
with open('Example.ipynb') as f:
    # nb = json.load(f)
    nb = nbformat.read(f, as_version=4)

In [5]:
cell = nb['cells'][2]

In [6]:
mb.is_flashcard(nb['cells'][2])

True

In [7]:
print(cell.source)

<!---->

**Example question goes here**

Some good and clear answer goes here

* maybe a bullet point
* or make it two
* add some dollar signs <span>\$</span>10 and <span>\$</span>20 usd
* bullet with $x = 4$ math and $x = 5$ more math

$$ x = 2^4 $$

$$ y = 1^3 $$

```python
def example_code():
    return 'hohoho'
```



In [8]:
meta, head, body = mb.process_cell_source(cell.source)
print('META:\n', meta, '\n')
print('HEAD:\n', head, '\n')
print('BODY:\n', body)

META:
 <!----> 

HEAD:
 **Example question goes here** 

BODY:
 
<div class="cell border-box-sizing text_cell rendered"><div class="prompt input_prompt">
</div><div class="inner_cell">
<div class="text_cell_render border-box-sizing rendered_html">
<p>Some good and clear answer goes here</p>
<ul>
<li>maybe a bullet point</li>
<li>or make it two</li>
<li>add some dollar signs $10 and $20 usd</li>
<li>bullet with \(x = 4\) math and \(x = 5\) more math</li>
</ul>
\[ x = 2^4 \]\[ y = 1^3 \]<div class="highlight"><pre><span></span><span class="k">def</span> <span class="nf">example_code</span><span class="p">():</span>
    <span class="k">return</span> <span class="s1">&#39;hohoho&#39;</span>
</pre></div>

</div>
</div>
</div>
 




In [9]:
from IPython.core.display import HTML

In [10]:
display(HTML(body))

# Anki Connect

Install:


1. Open the Install Add-on dialog by selecting Tools | Add-ons | Browse & Install in Anki.
2. Input **2055492159** into the text box labeled Code and press the OK button to proceed.
3. Restart Anki when prompted to do so in order to complete the installation of AnkiConnect.

Links:

* [AnkiConnect](https://foosoft.net/projects/anki-connect/) - website
* [anki-connect](https://github.com/FooSoft/anki-connect) - GitHub

In [11]:
import json
import urllib

In [12]:
def anki_invoke(action, **params):
    """Exec AnkiConnect RESTful querry.
    
    This method is low level REST API invocation, recommended to use wrappers instead
    
    For this to work, you have to:
     - have Anki installed with account ready and working, https://apps.ankiweb.net/
     - install AnkiConnect plugin, restart: https://foosoft.net/projects/anki-connect/
     - have Anki running on same computer in the background
    """
    
    def build_dict(action, **params):
        return {'action': action, 'params': params, 'version': 6}
    
    url = 'http://localhost:8765'
    payload_dict = build_dict(action, **params)
    payload_json = json.dumps(payload_dict).encode('utf-8')
    request = urllib.request.Request(url, payload_json)
    
    with urllib.request.urlopen(request) as f:
        response = json.load(f)
    
    if len(response) != 2:
        raise Exception('response has an unexpected number of fields')
    if 'error' not in response:
        raise Exception('response is missing required error field')
    if 'result' not in response:
        raise Exception('response is missing required result field')
    if response['error'] is not None:
        raise Exception(response['error'])
    return response['result']



In [14]:
result = anki_invoke('deckNamesAndIds')
display(result)

{'Taxonomy': 1529085506371,
 'Body - Training': 1530915734641,
 'Default': 1,
 'Master Deck': 1534961744134,
 'zz Input': 1535320252961,
 'zz Input 2': 1535682491436,
 'zz_Temp': 1542932513179,
 'Testing': 1559926834677}

In [24]:
result = anki_invoke('findNotes', query='deck:Testing')
display(result)

[1560017125593, 1560017138947]

In [16]:
result = anki_invoke('modelNamesAndIds')
display(result)

{'Basic (optional reversed card)': 1509323944264,
 'Basic (and reversed card)': 1509323944265,
 'Basic': 1509323944266,
 'Cloze': 1509323944263}

In [17]:
result = anki_invoke('modelFieldNames', modelName='Basic')
display(result)

['Front', 'Back']

In [25]:
note = {
    'deckName': 'Testing',
    'modelName': 'Basic',
    'fields': {
        'Front': 'front content',
        'Back': 'back content'
    },
    'options': { 'allowDuplicate': False },
    'tags': [],
}

In [26]:
result = anki_invoke('addNote', note=note)
display(result)

1560017298072

In [22]:
note = {
    'deckName': 'Testing',
    'modelName': 'Basic',
    'fields': {
        'Front': head,
        'Back': body
    },
    'options': { 'allowDuplicate': False },
    'tags': [],
}

In [23]:
result = anki_invoke('addNote', note=note)
display(result)

1560017138947